In [ ]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.is_leaf = False
        self.prediction = None

    def fit(self, X, y, depth=0):
        n_samples, n_features = X.shape

        # Stopping criteria
        if depth >= self.max_depth or len(np.unique(y)) == 1:
            self.is_leaf = True
            self.prediction = np.sign(np.sum(y))
            return

        # Find the best split
        best_feature, best_threshold, best_score = None, None, float("inf")
        for feature_i in range(n_features):
            thresholds = np.unique(X[:, feature_i])
            for threshold in thresholds:
                left_mask = X[:, feature_i] <= threshold
                right_mask = ~left_mask

                if len(np.unique(left_mask)) < 2:  # Prevent splits without valid groups
                    continue

                score = self._gini(y[left_mask], y[right_mask])
                if score < best_score:
                    best_feature, best_threshold, best_score = feature_i, threshold, score

        # If no valid split found
        if best_feature is None:
            self.is_leaf = True
            self.prediction = np.sign(np.sum(y))
            return

        # Create child nodes
        self.feature_index = best_feature
        self.threshold = best_threshold
        left_mask = X[:, self.feature_index] <= self.threshold
        right_mask = ~left_mask

        self.left = DecisionTree(self.max_depth)
        self.left.fit(X[left_mask], y[left_mask], depth + 1)

        self.right = DecisionTree(self.max_depth)
        self.right.fit(X[right_mask], y[right_mask], depth + 1)

    def predict(self, X):
        if self.is_leaf:
            return self.prediction

        if X[self.feature_index] <= self.threshold:
            return self.left.predict(X)
        else:
            return self.right.predict(X)

    def _gini(self, left_y, right_y):
        def gini_impurity(y):
            m = len(y)
            if m == 0:
                return 0
            p_pos = np.sum(y == 1) / m
            p_neg = np.sum(y == -1) / m
            return 1 - p_pos ** 2 - p_neg ** 2

        m = len(left_y) + len(right_y)
        return (len(left_y) / m) * gini_impurity(left_y) + (len(right_y) / m) * gini_impurity(right_y)


class RandomForest:
    def __init__(self, n_estimators=10, max_depth=3):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        for _ in range(self.n_estimators):
            # Bootstrap sampling
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_sample, y_sample = X[indices], y[indices]

            # Random feature subset
            feature_subset = np.random.choice(n_features, int(np.sqrt(n_features)), replace=False)
            X_sample = X_sample[:, feature_subset]

            # Train a decision tree
            tree = DecisionTree(max_depth=self.max_depth)
            tree.fit(X_sample, y_sample)
            self.trees.append((tree, feature_subset))

    def predict(self, X):
        tree_predictions = []
        for tree, features in self.trees:
            tree_predictions.append(np.array([tree.predict(x[features]) for x in X]))
        return np.sign(np.sum(tree_predictions, axis=0))

# Test the Random Forest
if __name__ == "__main__":
    # Dataset: features (x1, x2) and binary labels (1 or -1)
    X = np.array([
        [1, 2], [2, 3], [3, 1],
        [6, 5], [7, 8], [8, 6]
    ])
    y = np.array([-1, -1, -1, 1, 1, 1])  # Binary labels

    # Train the Random Forest
    forest = RandomForest(n_estimators=5, max_depth=3)
    forest.fit(X, y)

    # Predict on test data
    X_test = np.array([[2, 2], [7, 7]])
    predictions = forest.predict(X_test)
    print("Predictions:", predictions)

Predictions: [-1  1]
